In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv(r'S:\sonata\asssignment1\KOTAK\Final Files\PSF\los-merged-Account-Summary.csv',low_memory=False)
df2=df.copy()

In [8]:
n=df[['WRITE-OFF-AMT','OVERDUE-AMT']]
n.to_csv(r'S:\sonata\asssignment1\KOTAK\Final Files\PSF\powerbi.csv',index=False)

In [3]:
df['LAST-PAYMENT-DATE'].astype(str).str.strip()
df['LAST-PAYMENT-DATE']=pd.to_datetime(df['LAST-PAYMENT-DATE'], format='%Y-%m-%d', errors='coerce')

In [4]:
like_kotak=df['CREDIT-GRANTOR'].str.strip().str.upper().str.startswith('KOTAK')
not_kotak=(~df['CREDIT-GRANTOR'].str.strip().str.upper().str.startswith('KOTAK'))
like_jlg=df['ACCT-TYPE'].str.strip().str.upper().str.startswith('JLG')
acc_active=df['ACCOUNT-STATUS'].str.strip().str.upper()=='ACTIVE'
is_primary=df['MATCH-TYPE'].str.strip().str.upper()=='PRIMARY'
not_closed=df['ACCOUNT-STATUS'].str.strip().str.upper()!='CLOSED'
house_loan=df['ACCT-TYPE'].str.strip().str.upper()=='HOUSING LOAN'

In [5]:
df['NO_OwnMFIAssociation']=df['CREDIT-GRANTOR'].where(like_kotak & like_jlg & acc_active)
df['NO_OtherMFIAssociation']=df['CREDIT-GRANTOR'].where(not_kotak & like_jlg & acc_active)   
df['Overdue_Filtered']=pd.to_numeric(df['OVERDUE-AMT'].where(not_kotak & like_jlg & acc_active& not_closed & is_primary))
df['WriteOff_Amount']=pd.to_numeric(df['WRITE-OFF-AMT'].where(not_closed & not_kotak & like_jlg & is_primary))
df['WriteOff_Accounts']=df['WRITE-OFF-DATE'].where(not_closed & like_jlg & is_primary & not_kotak)
df['Housing_Loan_Installment']=pd.to_numeric(df['INSTALLMENT-AMT'].where(not_closed & house_loan &not_kotak).str.split('/').str[0])
df['Disbursed_Filtered']=df['DISBURSED-DT'].where( like_jlg & acc_active & not_kotak)
df['Last_Payment_Filtered']=df['LAST-PAYMENT-DATE'].where( like_jlg & acc_active & not_kotak)
df['MFI_Association_Rule']=df['CREDIT-GRANTOR'].where(not_kotak & like_jlg & acc_active & is_primary)
df['Overdue_Rule_Amount']=pd.to_numeric(df['OVERDUE-AMT'].where(not_kotak & like_jlg& not_closed & is_primary))
df['Customer_EMI']=pd.to_numeric(df['INSTALLMENT-AMT'].where(not_closed&not_kotak&is_primary).str.split('/').str[0] , errors='coerce').fillna(0).astype(int)
df['frequency']=df['INSTALLMENT-AMT'].where(not_closed&not_kotak&is_primary).str.split('/').str[1].str.strip().str.upper()

In [6]:
freq_map={
    'MONTHLY':1,
    ' BIWEEKLY':2,
    'QUARTERLY':0.33,
    'DAILY':30,
    'WEEKLY':4,
    'HALF YEARLY':0.166666,
    'HALF-YEARLY':0.166666,
    'ANNUALLY':0.015,
    'SEMI-ANNUALLY':0.015,
    'YEARLY':0.015,
    'OTHER':0.015
}

In [7]:
df['customerEmi']=df['frequency'].map(freq_map).fillna(0)

In [17]:
df['Disbursed_Filtered']=pd.to_datetime(df['Disbursed_Filtered'],format='%Y-%m-%d',errors='coerce')
df['Last_Payment_Filtered']=pd.to_datetime(df['Last_Payment_Filtered'],format='%Y-%m-%d',errors='coerce')
df['Customer_E']=df['customerEmi']*df['Customer_EMI']
im1=df.groupby('LOS-APP-ID').agg(
    NO_OwnMFIAssociation=('NO_OwnMFIAssociation','nunique'),
    NO_OtherMFIAssociation=('NO_OtherMFIAssociation','nunique'),
    Overdue_Amount=('Overdue_Filtered', 'sum'),
    Write_Off_Amount=('WriteOff_Amount', 'sum'),
    Write_Off_Accounts=('WriteOff_Accounts', 'nunique'),
    Housing_Loan=('Housing_Loan_Installment', 'sum'),
    Max_Disbursement_Date=('Disbursed_Filtered', 'max'),
    Max_Last_Payment_Date=('Last_Payment_Filtered', 'max'),
    MFI_AssociationRule_Rule=('MFI_Association_Rule', 'nunique'),
    Overdue_Rule=('Overdue_Rule_Amount', 'sum'),
    Customer_EMI=('Customer_E','sum'),
    max_Overdue_Amount=('Overdue_Filtered', 'max'),
    maxWrite_Off_Amount=('WriteOff_Amount', 'max')
).reset_index()

In [18]:
im2=df2.groupby('LOS-APP-ID').agg(
    PERFORMANCE_SCORE=('PERFORM_CNS SCORE','max')
)

In [19]:
im2['Score_rule']=im2['PERFORMANCE_SCORE'].apply(lambda x:1 if x>=645 else 0)

In [20]:
result=pd.merge(im1,im2 ,on='LOS-APP-ID')

In [21]:
result['Overdue_Amount']=result['Overdue_Amount'].apply(lambda x: 1 if x==0 else 0)
result['NO_OtherMFIAssociation']=result['NO_OtherMFIAssociation'].apply(lambda x: 1 if x<=3 else 0)
result['Write_Off_Amount']=result['Write_Off_Amount'].apply(lambda x: 1 if x==0 else 0)

In [23]:
final=result.where((result['NO_OtherMFIAssociation']==1)&(result['Write_Off_Amount']==1)&result['Overdue_Amount']==1)

In [24]:
final.dropna(inplace=True)

In [25]:
result.to_csv(r'S:\sonata\asssignment1\KOTAK\Final Files\PSF\assignment-2-analysis.csv',index=False)